This notebook explores advanced tools for improving the performances of your model. First, we will explore how to make better comparison with **cross validation** then what other models you can explore, and after the problem of **outliers** in this dataset, and finally how to **unskew** the different features. (These points are ranked by increasing order of difficulty).

In this notebook, every part is independent. So, if one does not sound interesting, just skip it !

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read the cleaned version
houses = pd.read_csv("data/")
kaggle = pd.read_csv("data/")

# Cross Validation

Is the model that we choose really the best ? Are you even sure that the way that we measure our performances is really fair ?

Because of the small size of our test set, we bias the estimation of how well our model is generalizing to new data ? To overcome this difficulty, we can explore a the **cross validation**. This method allows you to compute the performance of your model over the full dataset.

The idea is the following:
- Split your dataset into k groups
- Train on k-1 groups and test on the last
- Repeat k times (use each group as a test group once)

NB: $k <= NumberPoints$

![](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

Creates a loop that train and test the model for $k = 4$

Is there a significant impact of this measurement ?

# Other models ?

Why don't we try other simple models ! 

## Ridge Regression

The **ridge regression** is a linear regression which minimize the sum of the least square (the loss that you have previously coded) and a **regularizor** (in this case, the ridge regularizor which forces each parameters to be small)

In [ ]:
from sklearn.linear_model import Ridge

You can play with the parameter $\alpha$ that puts a more important weight on the regularizor (bigger it is, smaller is the chance to overfit, but it becomes possible to underfit - ie to not learn anything from the data :( )

## KNN

KNN stands for **k - nearest neighbors**, the idea is that, for each new points, you compute the k **closest points** and look at their labels (in our case, their prices) and you take the mean of these labels in order to make your predictions. The hyperparameter in this model is the number of neighbors (named `n_neighbors`) to take into account to have a strong good predictions.

The following picture shows for a new point (orange), which are its 5 closest neighbors (green) which will vote for the predictions. All the other blue points are the one that the model can use (for other example).

![](img/knn.png)

How would you improve this model ? 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

## SVM

**Support vector machines** allow to look at your data from another points of view. Sometimes it is not efficient to look at them in their original space, so **adding new dimensions** that are computed from the original data can improve the performances of the model.

The following example shows data which are in one dimension (on the left). Let's say that you have some points that have a negative label (orange) and all the other a positive one. In the orignal space it is not possible to separate them. 

Adding a new dimension (by using a particular function called kernel) allows to split the data thanks to a linear model: on the right it is easy to separate the data with a linear regression.

Original                   |Augmented
:-------------------------:|:-------------------------:
![](img/unclassifiable.png)|![](img/classifiable.png)

In [ ]:
from sklearn.svm import SVR

## Neural Network

And a last model, a neural network. A **perceptron** is the key component of a neural network. It is a simple weighted sum of its inputs, on which it applies a non linear function. You can repeat the number of perceptrons and organize them in layer as it is done in the following image (you can even multiple the number of hidden layers, which will make the model more complex). The training of this model is done thanks to a retropopagation of the error.

![](https://docs.google.com/drawings/d/e/2PACX-1vTiklLegD--T2-h9KZSmWZHk568vOdN44MHhlEQ4BHqtlvst-XlI5shHHyJhMCX5DdjuUaUFdmsz3t1/pub?w=327&h=223)

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
# More layers, longer will be the training
hidden_layer_sizes = [50, 75, 100] 

For each of the previous model, tries to find one advantage and disadvantage of the model.

What could be a good next step in order to improve your model ?

# Outliers

An outlier is a point that is distant from the rest of the datapoints. It may be due to measurement, experimental error or exception to the model. When it is an error, it is important to remove it from the dataset to avoid to learn misinformation.

In order to observe it, you can display the different houses and their price in a 2d plot for each feature. You will observe a central group and then some points far away from this distribution.

Create a for loop for displaying this 2d representation for every variables

In [ ]:
# To complete
for

However, you cannot just remove datapoints that look error. You want to achieve good performances on the test dataset, so you can analyze what are the range of the test data and remove the points that are outliers compare to this group. (You cannot remove points from testing, but you can limit the training set).

Plot the points for testing and training in a same plot

Should we remove any datapoints ? Do you have an idea to handle these misleading data without removing them ?

# Unskewed Feature

Do you remember, we have plot the distribution of the target variable and we weren't sure that it was normally distributed. Here is the statistical tools for exploring it.

First let's plot the distribution again

In [ ]:
# You have already coded this function, so I help you a little
def normalDistribution(points, mean, std):
    """
        Computes the normal distribution caracterized by mean and std
        at the given points
    """
    normal = 1/np.sqrt(2*np.pi*std**2) * np.exp(-(points - mean)**2/(2*std**2))
    return normal

The target variable looks right skewed. As (linear) models love normally distributed data , we need to transform this variable and make it more normally distributed.

The following function computes a test which will quantify if the distribution is close to a normal distribution. The test will return two values, the score and the **p-value** which quantify if we reject the hypothesis: the sample comes from a normal distribution. A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so you reject the null hypothesis.

So let's compute this value for the current distribution.

In [ ]:
from scipy.stats import normaltest

In [ ]:
def testNormal(variableName, sample, threshold = 0.05):
    print("Hypothesis: {} comes from a normal distribution."
          .format(variable))
    print(normaltest(sample).pvalue)
    if normaltest(sample).pvalue < threshold:
        print("\t=> This hypothesis can be rejected")
    else:
        print("\t=> This hypothesis cannot be rejected")

In [ ]:
testNormal("SalePrice", houses.SalePrice)

As (linear) models love normally distributed data, we will need to transform this variable and make it more normally distributed if we want to have a good model. The second statistical tool is the Box-Cox transformation which will transforms the sample to be closer to a normal distribution.
More information can be find in this [Stastistical Book](http://onlinestatbook.com/2/transformations/box-cox.html). (Don't be worried, it is advanced stats and you don't need to understand it to continue this project).

In [ ]:
from scipy.stats import boxcox

In [ ]:
correctedData = boxcox(houses.SalePrice)[0]

In [ ]:
# Displays the distribution of the new data
plt.figure()
sns.distplot(correctedData)
points = np.linspace(np.min(correctedData), np.max(correctedData), 10**4)
mean, std = np.mean(correctedData), np.std(correctedData)
plt.plot(points, normalDistribution(points, mean, std), color='red')
plt.show()

This is much better but if you use `testNormal`, the new distribution does not follow a normal distribution, but it is the best that we can do and the p-value has greatly increased.

You can know correct the distribution of each variables of the training set, this way the different features will have a distribution close to normal. Then you will have to apply the same transformation to the kaggle set (as we have done for the normalization).